In [1]:
###############################################################################################################################



###############################################################################################################################

# Analysis of Classification Efficiency of IMDB data using frquency bag-of-words

In [2]:
import re
import pandas as pd
import numpy as np

In [3]:
def TextPreprocess(text):   
    text = text.str.lower()
    text = text.str.replace('[^\w\s]','')    
    text = text.str.replace('[0-9]','')
    text = text.str.replace('_','')
    return text

In [4]:
vocab = pd.read_csv('IMDB-vocab.txt',header=None,names = ['word','id','frequency'])
#vocabtotext = vocab.word.str.cat(sep=' ')

In [5]:
#loc = 'D:/University materials/Winter 2018/Applied ML/winter 2018/Assignments/assignment 3/hwk3_datasets/IMDB-train.txt'
loc = 'IMDB-train.txt'
IMDB_train = pd.read_table(loc,header=None,names=['review','label'])
IMDB_train.review = TextPreprocess(IMDB_train.review)

In [6]:
#loc = 'D:/University materials/Winter 2018/Applied ML/winter 2018/Assignments/assignment 3/hwk3_datasets/IMDB-test.txt'
loc = 'IMDB-test.txt'
IMDB_test = pd.read_table(loc,header=None,names=['review','label'])
IMDB_test.review = TextPreprocess(IMDB_test.review)

In [7]:
#loc = 'D:/University materials/Winter 2018/Applied ML/winter 2018/Assignments/assignment 3/hwk3_datasets/IMDB-valid.txt'
loc = 'IMDB-valid.txt'
IMDB_valid = pd.read_table(loc,header=None,names=['review','label'])
IMDB_valid.review = TextPreprocess(IMDB_valid.review)

In [8]:
#############################################################################################################################
'''
One sprase matrix constructed save it and call the data when its needed as computation is time consuming
'''

############################################################################################################################

'\nOne sprase matrix constructed save it and call the data when its needed as computation is time consuming\n'

In [9]:
##########################################################################################################################



##########################################################################################################################

In [10]:
import scipy.sparse
train_mat = scipy.sparse.load_npz('IMDB_train_freq_mat.npz')
test_mat = scipy.sparse.load_npz('IMDB_test_freq_mat.npz')
valid_mat = scipy.sparse.load_npz('IMDB_valid_freq_mat.npz')


In [11]:
#train_set = yelp_train.iloc[np.random.choice(len(yelp_train.review),20)]       # randomly picking dataset to train
#validation_set = yelp_valid.iloc[np.random.choice(len(yelp_valid.review),20)]      # randomly picking dataset to classify

In [12]:
train_y_true = IMDB_train.label
valid_y_true = IMDB_valid.label
test_y_true = IMDB_test.label

### Applying Naive bayes classifier

In [13]:
# computeing accuracy and F1 score for predicting "training data"
# input sparse matrix data that needs to be predicted
def getClassifierEff (Data,true_y,clf):
    from sklearn.metrics import f1_score,accuracy_score
    y_pred = clf.predict(Data)
    acc = accuracy_score(true_y,y_pred)
    f1 = f1_score(true_y,y_pred,average='micro')
    return f1,acc

In [14]:
# training classifier with "training data"
from sklearn.naive_bayes import MultinomialNB

In [15]:
Hyp = [0, 1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
IMDB_valid_f1 = []
IMDB_valid_acc = []
for itr in range(len(Hyp)):
    naive_clf = MultinomialNB(alpha=Hyp[itr]).fit(train_mat, train_y_true)
    f1 , acc = getClassifierEff (valid_mat,valid_y_true,naive_clf)
    IMDB_valid_f1.append(f1)
    IMDB_valid_acc.append(acc)
   # print('IMDB_valid_f1',IMDB_valid_f1)

d:\machine learning\python 3.5.2\lib\site-packages\sklearn\naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


In [16]:
optim_alpha = Hyp[np.argmax(IMDB_valid_f1)]
print('for hyper parameter alpha =',Hyp[np.argmax(IMDB_valid_f1)],'we get max F1 score')

for hyper parameter alpha = 0.01 we get max F1 score


In [33]:
naive_clf = MultinomialNB( alpha=optim_alpha ).fit(train_mat, train_y_true)

train_f1 , train_acc = getClassifierEff (train_mat,train_y_true,naive_clf)
test_f1 , test_acc = getClassifierEff (test_mat,test_y_true,naive_clf)
valid_f1 , valid_acc = getClassifierEff (valid_mat,valid_y_true,naive_clf)

print('IMDB_train_f1',train_f1)
print('IMDB_test_f1',test_f1)
print('IMDB_valid_f1',valid_f1)


IMDB_train_f1 0.8598
IMDB_test_f1 0.81252
IMDB_valid_f1 0.8288


In [19]:
from sklearn.naive_bayes import GaussianNB

In [20]:
IMDB_valid_f1 = []
IMDB_valid_acc = []

naive_clf = GaussianNB().fit(train_mat.toarray(), train_y_true)
f1 , acc = getClassifierEff (valid_mat.toarray(),valid_y_true,naive_clf)
IMDB_valid_f1 = f1
IMDB_valid_acc = acc

train_f1 , train_acc = getClassifierEff (train_mat.toarray(),train_y_true,naive_clf)
test_f1 , test_acc = getClassifierEff (test_mat.toarray(),test_y_true,naive_clf)

MemoryError: 

### Applying Decision Tree

In [21]:
from sklearn import tree

class sklearn.tree.DecisionTreeClassifier(criterion=’gini’, splitter=’best’, max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort=False)

In [22]:
crit = ['gini','entropy'] #criterion 
split = ['best','random'] #splitter


In [23]:
IMDB_valid_f1 = []
IMDB_valid_acc = []
for itr1 in range(len(crit)):
    for itr2 in range(len(split)):
        tree_clf = tree.DecisionTreeClassifier(criterion = crit[itr1], splitter = split[itr2]).fit(train_mat, train_y_true)
        f1,acc = getClassifierEff (valid_mat,valid_y_true,tree_clf)
        IMDB_valid_f1.append(f1)
        IMDB_valid_acc.append(acc)
        
        print('IMDB_valid_f1',f1,'when we use criterion',crit[itr1],'and splitter',split[itr2])

IMDB_valid_f1 0.6971 when we use criterion gini and splitter best
IMDB_valid_f1 0.7033 when we use criterion gini and splitter random
IMDB_valid_f1 0.6972 when we use criterion entropy and splitter best
IMDB_valid_f1 0.7079000000000001 when we use criterion entropy and splitter random


In [24]:
tree_clf = tree.DecisionTreeClassifier(criterion = 'entropy', splitter = 'best').fit(train_mat, train_y_true)
IMDB_train_f1,IMDB_train_acc = getClassifierEff (train_mat,train_y_true,tree_clf)
IMDB_test_f1,IMDB_test_acc = getClassifierEff (test_mat,test_y_true,tree_clf)

print('IMDB_train_f1',IMDB_train_f1)
print('IMDB_test_f1',IMDB_test_f1)

IMDB_train_f1 1.0
IMDB_test_f1 0.69984


### Applying Linear SVC

class sklearn.svm.LinearSVC(penalty=’l2’, loss=’squared_hinge’, dual=True, tol=0.0001, C=1.0, multi_class=’ovr’, fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0, random_state=None, max_iter=1000)

In [25]:
from sklearn.svm import LinearSVC
linear_clf = LinearSVC().fit(train_mat, train_y_true)

### for combination of penalty='l2' , loss='hinge',dual=True

In [27]:
pen = ['l2']
los = ['hinge'] # gives error for hinge
dul = [True] # gives error for dual = true. 
tolerance = [1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1,1]
C_param = [1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1,1]

In [28]:
IMDB_valid_f1 = []
IMDB_valid_acc = []
for itr1 in range(len(pen)):

    linear_clf = LinearSVC(penalty='l2' , loss='hinge', dual=True).fit(train_mat, train_y_true)
    f1 , acc = getClassifierEff (valid_mat,valid_y_true,linear_clf)
    IMDB_valid_f1.append(f1)
    IMDB_valid_acc.append(acc)
            
    print('IMDB_valid_f1',f1,'when we use penalty',pen[itr1],'loss',los[itr1],'dual',dul[itr1])

IMDB_valid_f1 0.8427 when we use penalty l2 loss hinge dual True


In [29]:
IMDB_valid_f1 = []
IMDB_valid_acc = []
for itr1 in range(len(pen)):
    for itr2 in range(len(tolerance)):
        for itr3 in range(len(C_param)):
            linear_clf = LinearSVC(penalty=pen[itr1],tol=tolerance[itr2],C=C_param[itr3],dual=False).fit(train_mat, train_y_true)
            f1 , acc = getClassifierEff (valid_mat,valid_y_true,linear_clf)
            IMDB_valid_f1.append(f1)
            IMDB_valid_acc.append(acc)
            
            print('IMDB_valid_f1',f1,'when we use penalty',pen[itr1],'tolerence',tolerance[itr2],'C',C_param[itr3])

IMDB_valid_f1 0.5076 when we use penalty l2 tolerence 1e-10 C 1e-10
IMDB_valid_f1 0.5081 when we use penalty l2 tolerence 1e-10 C 1e-09
IMDB_valid_f1 0.518 when we use penalty l2 tolerence 1e-10 C 1e-08
IMDB_valid_f1 0.5849 when we use penalty l2 tolerence 1e-10 C 1e-07
IMDB_valid_f1 0.6623 when we use penalty l2 tolerence 1e-10 C 1e-06
IMDB_valid_f1 0.736 when we use penalty l2 tolerence 1e-10 C 1e-05
IMDB_valid_f1 0.8303 when we use penalty l2 tolerence 1e-10 C 0.0001
IMDB_valid_f1 0.8727999999999999 when we use penalty l2 tolerence 1e-10 C 0.001
IMDB_valid_f1 0.8778 when we use penalty l2 tolerence 1e-10 C 0.01
IMDB_valid_f1 0.8606 when we use penalty l2 tolerence 1e-10 C 0.1
IMDB_valid_f1 0.8486 when we use penalty l2 tolerence 1e-10 C 1
IMDB_valid_f1 0.5076 when we use penalty l2 tolerence 1e-09 C 1e-10
IMDB_valid_f1 0.5081 when we use penalty l2 tolerence 1e-09 C 1e-09
IMDB_valid_f1 0.518 when we use penalty l2 tolerence 1e-09 C 1e-08
IMDB_valid_f1 0.5849 when we use penalty l2 t

In [30]:
linear_clf = LinearSVC(penalty='l2',tol=0.01,C=0.01,loss='hinge',dual=True).fit(train_mat, train_y_true)
IMDB_train_f1,IMDB_train_acc = getClassifierEff (train_mat,train_y_true,linear_clf)
IMDB_test_f1,IMDB_test_acc = getClassifierEff (test_mat,test_y_true,linear_clf)
IMDB_valid_f1,IMDB_valid_acc = getClassifierEff (valid_mat,valid_y_true,linear_clf)
print('IMDB_train_f1',IMDB_train_f1)
print('IMDB_test_f1',IMDB_test_f1)
print('IMDB_valid_f1',IMDB_valid_f1)

IMDB_train_f1 0.9415333333333333
IMDB_test_f1 0.87272
IMDB_valid_f1 0.8796999999999999


### for combination of penalty='l1','l2' , loss='squred_hinge',dual=False

In [31]:
pen = ['l1','l2']
los = ['squared_hinge'] # gives error for hinge
dul = [False] # gives error for dual = true. 
tolerance = [1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
C_param = [1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]

In [32]:
IMDB_valid_f1 = []
IMDB_valid_acc = []
for itr1 in range(len(pen)):
    for itr2 in range(len(tolerance)):
        for itr3 in range(len(C_param)):
            linear_clf = LinearSVC(penalty=pen[itr1],tol=tolerance[itr2],C=C_param[itr3],dual=False).fit(train_mat, train_y_true)
            f1 , acc = getClassifierEff (valid_mat,valid_y_true,linear_clf)
            IMDB_valid_f1.append(f1)
            IMDB_valid_acc.append(acc)
            
            print('IMDB_valid_f1',f1,'when we use penalty',pen[itr1],'tolerence',tolerance[itr2],'C',C_param[itr3])

IMDB_valid_f1 0.5 when we use penalty l1 tolerence 1e-10 C 1e-10
IMDB_valid_f1 0.5 when we use penalty l1 tolerence 1e-10 C 1e-09
IMDB_valid_f1 0.5 when we use penalty l1 tolerence 1e-10 C 1e-08
IMDB_valid_f1 0.5 when we use penalty l1 tolerence 1e-10 C 1e-07
IMDB_valid_f1 0.5 when we use penalty l1 tolerence 1e-10 C 1e-06
IMDB_valid_f1 0.5 when we use penalty l1 tolerence 1e-10 C 1e-05
IMDB_valid_f1 0.5256 when we use penalty l1 tolerence 1e-10 C 0.0001
IMDB_valid_f1 0.7292999999999998 when we use penalty l1 tolerence 1e-10 C 0.001
IMDB_valid_f1 0.8461 when we use penalty l1 tolerence 1e-10 C 0.01
IMDB_valid_f1 0.8694 when we use penalty l1 tolerence 1e-10 C 0.1
IMDB_valid_f1 0.5 when we use penalty l1 tolerence 1e-09 C 1e-10
IMDB_valid_f1 0.5 when we use penalty l1 tolerence 1e-09 C 1e-09
IMDB_valid_f1 0.5 when we use penalty l1 tolerence 1e-09 C 1e-08
IMDB_valid_f1 0.5 when we use penalty l1 tolerence 1e-09 C 1e-07
IMDB_valid_f1 0.5 when we use penalty l1 tolerence 1e-09 C 1e-06
IMD

IMDB_valid_f1 0.5081 when we use penalty l2 tolerence 1e-08 C 1e-09
IMDB_valid_f1 0.518 when we use penalty l2 tolerence 1e-08 C 1e-08
IMDB_valid_f1 0.5849 when we use penalty l2 tolerence 1e-08 C 1e-07
IMDB_valid_f1 0.6623 when we use penalty l2 tolerence 1e-08 C 1e-06
IMDB_valid_f1 0.736 when we use penalty l2 tolerence 1e-08 C 1e-05
IMDB_valid_f1 0.8303 when we use penalty l2 tolerence 1e-08 C 0.0001
IMDB_valid_f1 0.8727999999999999 when we use penalty l2 tolerence 1e-08 C 0.001
IMDB_valid_f1 0.8778 when we use penalty l2 tolerence 1e-08 C 0.01
IMDB_valid_f1 0.8606 when we use penalty l2 tolerence 1e-08 C 0.1
IMDB_valid_f1 0.5076 when we use penalty l2 tolerence 1e-07 C 1e-10
IMDB_valid_f1 0.5081 when we use penalty l2 tolerence 1e-07 C 1e-09
IMDB_valid_f1 0.518 when we use penalty l2 tolerence 1e-07 C 1e-08
IMDB_valid_f1 0.5849 when we use penalty l2 tolerence 1e-07 C 1e-07
IMDB_valid_f1 0.6623 when we use penalty l2 tolerence 1e-07 C 1e-06
IMDB_valid_f1 0.736 when we use penalty l

In [35]:
linear_clf = LinearSVC(penalty='l2',tol=0.01,C=0.01,loss='squared_hinge',dual=False).fit(train_mat, train_y_true)
IMDB_train_f1,IMDB_train_acc = getClassifierEff (train_mat,train_y_true,linear_clf)
IMDB_test_f1,IMDB_test_acc = getClassifierEff (test_mat,test_y_true,linear_clf)
IMDB_valid_f1,IMDB_valid_acc = getClassifierEff (valid_mat,valid_y_true,linear_clf)
print('IMDB_train_f1',IMDB_train_f1)
print('IMDB_test_f1',IMDB_test_f1)
print('IMDB_valid_f1',IMDB_valid_f1)

IMDB_train_f1 0.9715333333333334
IMDB_test_f1 0.86928
IMDB_valid_f1 0.8778
